In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics

2017-06-16 19:49:32,992 : INFO : 'pattern' package found; tag filters are available for English
/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Model 1 evaluation

In [2]:
model = gensim.models.Word2Vec.load('/home/bahbbc/workspace/masters-big5/models/tweet90.model')
model.init_sims(replace=True)

2017-06-16 19:49:33,532 : INFO : loading Word2Vec object from /home/bahbbc/workspace/masters-big5/models/tweet90.model
2017-06-16 19:49:44,488 : INFO : loading wv recursively from /home/bahbbc/workspace/masters-big5/models/tweet90.model.wv.* with mmap=None
2017-06-16 19:49:44,489 : INFO : loading syn0 from /home/bahbbc/workspace/masters-big5/models/tweet90.model.wv.syn0.npy with mmap=None
2017-06-16 19:50:07,070 : INFO : setting ignored attribute syn0norm to None
2017-06-16 19:50:07,090 : INFO : loading syn1neg from /home/bahbbc/workspace/masters-big5/models/tweet90.model.syn1neg.npy with mmap=None
2017-06-16 19:51:06,373 : INFO : setting ignored attribute cum_table to None
2017-06-16 19:51:07,219 : INFO : loaded /home/bahbbc/workspace/masters-big5/models/tweet90.model
2017-06-16 19:53:31,160 : INFO : precomputing L2-norms of word weight vectors


In [3]:
print(model.most_similar(positive=['rss', 'haha']))

[(u'hehe', 0.9164583683013916), (u'hahah', 0.9039794206619263), (u'hahaha', 0.8907245397567749), (u'hehehe', 0.8794063329696655), (u'rs', 0.8751157522201538), (u'kk', 0.8694721460342407), (u'rsrs', 0.8604087829589844), (u'ahaha', 0.8547937870025635), (u'kkk', 0.8498312830924988), (u'hahahah', 0.844302773475647)]


In [4]:
print(model.most_similar(positive=[':D', ':/']))

[(u':p', 0.8323384523391724), (u':(', 0.8253865242004395), (u'^^', 0.8235442638397217), (u':))', 0.8181711435317993), (u':P', 0.8121601343154907), (u':s', 0.8120744228363037), (u':\\', 0.8096354007720947), (u';/', 0.8064714670181274), (u'://', 0.8019497394561768), (u':)', 0.8007704019546509)]


In [5]:
print(model.most_similar(positive=['fofo', 'cute']))

[(u'fofinho', 0.7538474798202515), (u'meigo', 0.7201195955276489), (u'fofis', 0.711998701095581), (u'fofo,', 0.690879762172699), (u'lindinho', 0.6813713908195496), (u'kawaii', 0.6526966094970703), (u'foufo', 0.646642804145813), (u'adorable', 0.6454997062683105), (u'fofo!', 0.6454809904098511), (u'fofoooo', 0.6446171998977661)]


In [6]:
num_features= 300

In [7]:
df = pd.read_csv('~/personality-normalized-word2vec-norm.csv', encoding='utf-8')
df.shape

(1039, 186)

In [8]:
df.extraversion_m.value_counts()

0    529
1    510
Name: extraversion_m, dtype: int64

In [9]:
train_w2v_data, test_w2v_data = train_test_split(df, test_size=0.5, random_state=42)

In [10]:
def predict(vectorizer, classifier, data):
    data_features = vectorizer.transform(data['formatted_text'])
    predictions = classifier.predict(data_features)
    target = data['extraversion_m']
    evaluate_prediction(predictions, target)

In [11]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0norm[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(num_features,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [12]:
def w2v_tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [13]:
test_tokenized = test_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values
train_tokenized = train_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values

In [14]:
%%time
X_train_word_average = word_averaging_list(model,train_tokenized)
X_test_word_average = word_averaging_list(model,test_tokenized)

2017-06-16 19:55:16,292 : WARNING : cannot compute similarity with no input []
2017-06-16 19:55:16,316 : WARNING : cannot compute similarity with no input []
2017-06-16 19:55:17,956 : WARNING : cannot compute similarity with no input []
2017-06-16 19:55:18,398 : WARNING : cannot compute similarity with no input []
2017-06-16 19:55:19,039 : WARNING : cannot compute similarity with no input []
2017-06-16 19:55:20,042 : WARNING : cannot compute similarity with no input []
2017-06-16 19:55:20,147 : WARNING : cannot compute similarity with no input []
2017-06-16 19:55:20,458 : WARNING : cannot compute similarity with no input []
2017-06-16 19:55:20,677 : WARNING : cannot compute similarity with no input []
2017-06-16 19:55:20,753 : WARNING : cannot compute similarity with no input []
2017-06-16 19:55:22,095 : WARNING : cannot compute similarity with no input []


CPU times: user 5.06 s, sys: 386 ms, total: 5.45 s
Wall time: 10.5 s


In [15]:
del model

In [16]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

force = False
model_trainer = RandomizedSearchCV(
    n_iter=1, 
    estimator=RandomForestClassifier(),
    param_distributions={
        "criterion": ["gini"],
        "n_estimators": [1000],
        "max_features": ["log2"],
        "max_depth": [None],
        "bootstrap": [True],
        "oob_score": [True],
        "class_weight": ["balanced"],
    },
    scoring="f1",
    verbose=True,
    refit=True,
    cv=10,
    n_jobs=-1
)

In [17]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['extraversion_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   23.1s finished


CPU times: user 4.57 s, sys: 449 ms, total: 5.02 s
Wall time: 30 s


In [18]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['extraversion_m']

In [19]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,164,85
1,134,137


In [20]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.55      0.66      0.60       249
          1       0.62      0.51      0.56       271

avg / total       0.59      0.58      0.58       520



In [21]:
accuracy_score(yt, yp)

0.5788461538461539

In [22]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [23]:
ytp = model.predict(X_train_word_average)
ytt = train_w2v_data['extraversion_m']

In [24]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       280
          1       1.00      1.00      1.00       239

avg / total       1.00      1.00      1.00       519



In [25]:
print accuracy_score(ytt, ytp)

1.0
